In [50]:
import pandas as pd
import matplotlib.pyplot as plt

In [51]:
full_file = pd.read_csv("albumlist.csv", encoding='latin-1')
full_file.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [52]:
grouped_by_Genre= full_file.groupby(["Artist"]) .count()
grouped_by_Genre

,Number,Year,Album,Genre,Subgenre
Artist,,,,,
A Tribe Called Quest,1,1,1,1,1
ABBA,1,1,1,1,1
AC/DC,2,2,2,2,2
Aerosmith,2,2,2,2,2
Al Green,3,3,3,3,3
...,...,...,...,...,...
Willie Nelson,2,2,2,2,2
Wire,1,1,1,1,1
Wu Tang Clan,1,1,1,1,1


In [55]:
full_file.head(remove_categories("subgenre"))

NameError: name 'remove_categories' is not defined

In [53]:
full_file.remove_categories(["Subgenre"])


AttributeError: 'DataFrame' object has no attribute 'remove_categories'